# Use Keras Models With Scikit-Learn For General Machine Learning

<a href="https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/">Chapter 9</a>

## Evaluate Models with Cross Validation

In [6]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

# Function to create model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Input(shape=(8,)))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load pima indians dataset
dataset = np.loadtxt("datasets/pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=10, verbose=1)

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Epoch 1/150
70/70 [==============================] - 2s 3ms/step - loss: 13.3902 - accuracy: 0.6512
Epoch 2/150
70/70 [==============================] - 0s 2ms/step - loss: 4.8376 - accuracy: 0.6512
Epoch 3/150
70/70 [==============================] - 0s 3ms/step - loss: 1.1997 - accuracy: 0.5007
Epoch 4/150
70/70 [==============================] - 0s 2ms/step - loss: 0.8795 - accuracy: 0.5499
Epoch 5/150
70/70 [==============================] - 0s 2ms/step - loss: 0.7906 - accuracy: 0.6064
Epoch 6/150
70/70 [==============================] - 0s 3ms/step - loss: 0.7392 - accuracy: 0.6194
Epoch 7/150
70/70 [==============================] - 0s 4ms/step - loss: 0.7256 - accuracy: 0.6411
Epoch 8/150
70/70 [==============================] - 0s 3ms/step - loss: 0.7165 - accuracy: 0.6165
Epoch 9/150
70/70 [==============================] - 0s 3ms/step - loss: 0.6967 - accuracy: 0.6570
Epoch 10/150
70/70 [==============================] - 0s 3ms/step - loss: 0.6908 - accuracy: 0.6498
Epoch 11

**In comparison, the following is an equivalent implementation with a neural network model in scikit-learn:**

In [8]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
import numpy as np

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load pima indians dataset
dataset = np.loadtxt("datasets/pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = MLPClassifier(hidden_layer_sizes=(12,8), activation='relu', max_iter=150, batch_size=10, verbose=False)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

C:\Users\shayn\PycharmProjects\DeepLearning_MLM\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7123547505126452


# Grid Search Deep Learning Model Parameters

In [9]:
# MLP for Pima Indians Dataset with grid search via sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Input(shape=(8, )))
    model.add(Dense(12,  kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
 
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load pima indians dataset
dataset = np.loadtxt("datasets/pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, verbose=0)
print(model.get_params().keys())

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, model__init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'class_weight'])
Best: 0.769544 using {'batch_size': 10, 'epochs': 150, 'model__init': 'normal', 'optimizer': 'adam'}
0.691486 (0.034116) with: {'batch_size': 5, 'epochs': 50, 'model__init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.694024 (0.038370) with: {'batch_size': 5, 'epochs': 50, 'model__init': 'glorot_uniform', 'optimizer': 'adam'}
0.735761 (0.034932) with: {'batch_size': 5, 'epochs': 50, 'model__init': 'normal', 'optimizer': 'rmsprop'}
0.710958 (0.037682) with: {'batch_size': 5, 'epochs': 50, 'model__init': 'normal', 'optimizer': 'adam'}
0.725244 (0.015879) with: {'batch_size': 5, 'epochs': 50, 'model__init': 'uniform', 'optimizer': 'rmsprop'}
0.708344 (0.015928) with: {'batch_size': 5, 'epochs': 50, 'model__init': 'uniform', 'optimizer': 'adam'}
0.730532 (0.025566